In [ ]:
import pandas as pd

In [ ]:
dataset_name = 'sidra_tab1092_bovino'
raiz = '../databases'

### Bronze

In [ ]:
bronze_df = (
    pd.read_json('https://apisidra.ibge.gov.br/values/t/1092/n3/all/v/285/p/199701-202504/c12716/115236/c18/992/c12529/118225')
)

bronze_df.to_json(f'{raiz}/bronze_{dataset_name}_uf.json')

# (pd.read_json('https://apisidra.ibge.gov.br/values/t/1092/n1/all/v/285/p/last%2094/c12716/115236/c18/992/c12529/118225')
#  .to_json(f'{raiz}/bronze_{dataset_name}_BR.json')
#  )

bronze_df.head()

### Prata

In [ ]:
silver_df = (
    pd.read_json(f'{raiz}/bronze_{dataset_name}_uf.json')
        .filter(['D1C', 'D3C', 'V'])
        .drop(0, axis = 0)
        .assign(**{
            'ano': lambda tri: tri['D3C'].str.extract('(\\d{4})').astype(int),
            'peso_kg': lambda df: pd.to_numeric(df['V'], errors='coerce'),
            'uf': lambda df: df['D1C'].astype(int),
            'fonte': 'Carcaça Bovina'
        })
        .dropna(subset='peso_kg')
        .groupby(['uf', 'ano', 'fonte'])
        .agg({'peso_kg': sum}) 
 )

silver_df.to_parquet(f'{raiz}/prata_{dataset_name}.parquet')

silver_df.head()

In [ ]:
# df_br = (pd.read_json(f'{raiz}/bronze_{dataset_name}_BR.json')
#  .filter(['D1C', 'D3C', 'V'])
#  .drop(0, axis = 0)
#  .assign(**{'ano': lambda tri: tri['D3C'].str.extract('(\\d{4})').astype(int),
#             'peso_kg': lambda df: pd.to_numeric(df['V'], errors='coerce'),
#             'uf': lambda df: df['D1C'].astype(int),
#             'fonte': 'Carcaça Bovina'
#             })
#  .groupby(['uf', 'ano', 'fonte'])
#  .agg({'peso_kg': sum})
#  )

### Ouro

Peso total de carcaças dos bovinos abatidos (kg)

Aproveitamento da carcaça: 50%

Cálculo: ((Carcaça Bovina/aproveitamento%) * (Parâmetro_x%))/1000

UF e Brasil = t

NPK na forma de Carne bovina retirada do solo via animal encaminhado para abate (t)

In [ ]:
aproveitamento = 50/100

parametro_N = 3/100
parametro_P = 0.8/100
parametro_K = 0.2/100

In [ ]:
# valor em tonelada
gold_df = (
    pd.read_parquet(f'{raiz}/prata_{dataset_name}.parquet')
        .assign(**{
            'N': lambda df: ((df['peso_kg']/aproveitamento)*parametro_N)/1000,
            'P': lambda df: ((df['peso_kg']/aproveitamento)*parametro_P)/1000,
            'K': lambda df: ((df['peso_kg']/aproveitamento)*parametro_K)/1000
        })
        .drop(
            "peso_kg", axis="columns"
        )
        .stack()
        .rename_axis(("uf", "ano", "fonte", "elemento"), axis="rows")
        .rename("valor")
        .sort_index()
      # .reset_index()
      # .melt(id_vars=['uf', 'ano', 'fonte'],
      #       value_vars=['N', 'P', 'K'],
      #       var_name='elemento',
      #       value_name='valor')
      # .convert_dtypes()
      # .set_index(['uf', 'ano', 'fonte'])
)

gold_df.to_csv(f'{raiz}/ouro_npk_carcaca_bovina.csv')

gold_df